<a href="https://colab.research.google.com/github/shardabhavya/Coursera_Capstone/blob/master/Mumbai_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Moving to Mumbai**
This project aims at exploring Mumbai as a business opportunity for a chinese restaurant, what are the areas one should prefer to open up a restaurant.

Data Collection for Project

Folium library to display the map of city For fetching data we will use following

Foursquare API: To fetch the venues in Mumbai.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #for plotting graphs
import folium #for plotting maps
import requests #Library to handle requests
from pandas.io.json import json_normalize #Transfer JSON file into pandas dataframe
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Loading Mumbai Neighbourhood Dataset found Online!

In [3]:
df_mumbai = pd.read_csv("Mumbai.csv")

In [4]:
df_mumbai.head()

,Location,Pincode,State,District,Unnamed: 4
0,A I staff colony,400029,Maharashtra,Mumbai,NaN
1,Aareymilk Colony,400065,Maharashtra,Mumbai,NaN
2,Agripada,400011,Maharashtra,Mumbai,NaN
3,Airport,400099,Maharashtra,Mumbai,NaN
4,Ambewadi,400004,Maharashtra,Mumbai,NaN


In [5]:
df_mumbai.drop(columns= 'Unnamed: 4', inplace=True)
df_mumbai.head()

,Location,Pincode,State,District
0,A I staff colony,400029,Maharashtra,Mumbai
1,Aareymilk Colony,400065,Maharashtra,Mumbai
2,Agripada,400011,Maharashtra,Mumbai
3,Airport,400099,Maharashtra,Mumbai
4,Ambewadi,400004,Maharashtra,Mumbai


Fetching Venues using Foursqure API, Foursqured Details are:

In [6]:
CLIENT_ID = 'NRXSZTZGJ3J5EKINI4AFJ3XS3PFRXS32AOCJ33CL0FKALKEN' #Client ID
CLIENT_SECRET = '2R2BCKZKES0NBC3OPKO2ALC1XOXVP1YDLFJZ21P44HMQ0FZZ' #Client Secret
VERSION = '20200825'#Current Date
RADIUS = 1000 #KM
LIMIT = 500 #No. of Venues

Fetching Latitude and Longitudes of each neighbourhood in Mumbai using Nominatim Geocode!

In [7]:
myAddress = df_mumbai['Location']

locator = Nominatim(user_agent="myGeocoder")

loc_latitude = []
loc_longitude = []

for myA in myAddress:
    try:
        myA = myA + ", Mumbai"
        location = locator.geocode(myA)
        if location == None:
          Lat = 'NaN'
          Lng = 'NaN'
        else:
          Lat = location.latitude
          Lng = location.longitude
        loc_latitude.append(Lat)
        #print('loc_latitude',loc_latitude)
        loc_longitude.append(Lng)
    except Exception as e:
        print('error:::',e)                                                                                    
df_mumbai['Latitude']= loc_latitude
df_mumbai['Longitude']= loc_longitude
df_mumbai

,Location,Pincode,State,District,Latitude,Longitude
0,A I staff colony,400029,Maharashtra,Mumbai,NaN,NaN
1,Aareymilk Colony,400065,Maharashtra,Mumbai,NaN,NaN
2,Agripada,400011,Maharashtra,Mumbai,18.9753,72.8249
3,Airport,400099,Maharashtra,Mumbai,19.0901,72.8637
4,Ambewadi,400004,Maharashtra,Mumbai,19.1868,72.8593
...,...,...,...,...,...,...
177,Worli,400018,Maharashtra,Mumbai,19.0117,72.8181
178,Worli Colony,400030,Maharashtra,Mumbai,18.9752,72.824
179,Worli Naka,400018,Maharashtra,Mumbai,18.987,72.8285
180,Worli Police camp,400030,Maharashtra,Mumbai,19.0098,72.8172


Removing NaN values from Table!

In [8]:
df_mumbai2 = df_mumbai[df_mumbai.Latitude != 'NaN']
df_mumbai2

,Location,Pincode,State,District,Latitude,Longitude
2,Agripada,400011,Maharashtra,Mumbai,18.9753,72.8249
3,Airport,400099,Maharashtra,Mumbai,19.0901,72.8637
4,Ambewadi,400004,Maharashtra,Mumbai,19.1868,72.8593
5,Andheri,400053,Maharashtra,Mumbai,19.1197,72.8464
6,Andheri East,400069,Maharashtra,Mumbai,19.1159,72.8542
...,...,...,...,...,...,...
177,Worli,400018,Maharashtra,Mumbai,19.0117,72.8181
178,Worli Colony,400030,Maharashtra,Mumbai,18.9752,72.824
179,Worli Naka,400018,Maharashtra,Mumbai,18.987,72.8285
180,Worli Police camp,400030,Maharashtra,Mumbai,19.0098,72.8172


Plotting these Neighbourhoods on Map using Folium!

In [11]:
#latitude = df_mumbai2.loc['Latitude']
#longitude = df_mumbai2.loc['Longitude']
latitude = 19.0760
longitude = 72.8777
mumbai_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_mumbai2['Latitude'],df_mumbai2['Longitude'],df_mumbai2['Location']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mumbai_map)  
    
mumbai_map

In [12]:
venues_list = []
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            19.0760, 
            72.8777, 
            RADIUS, 
            LIMIT)
  
results = requests.get(url).json()

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
results

{'meta': {'code': 200, 'requestId': '5f49d2955c3dee61cc345734'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54d38a72498e7d4deee4e4c4-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d116941735',
         'name': 'Bar',
         'pluralName': 'Bars',
         'primary': True,
         'shortName': 'Bar'}],
       'id': '54d38a72498e7d4deee4e4c4',
       'location': {'address': 'BKC',
        'cc': 'IN',
        'city': 'Mumbai',
        'country': 'India',
        'distance': 556,
        'formattedAddress': ['BKC', 'Mumbai', 'Mahārāshtra', 'India'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 19.071165828845604,
          'lng': 72.87635864580997}],
        'lat': 19.0711

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Bar Stock Exchange,Bar,19.071166,72.876359
1,Delhi Zaika,Indian Restaurant,19.077054,72.878260
2,Sahara Restaurant,Mughlai Restaurant,19.079532,72.880152
3,Pizza Hut,Pizza Place,19.075984,72.877656
4,24 Shisha Lounge,Lounge,19.072410,72.870194


In [16]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
Mumbai_venues = getNearbyVenues(names=df_mumbai2['Location'],
                                   latitudes=df_mumbai2['Latitude'],
                                   longitudes=df_mumbai2['Longitude']
                                  )

Agripada
Airport
Ambewadi
Andheri
Andheri East
Andheri Railway station
Antop Hill
Asvini
Azad Nagar
B P t colony
B.N. bhavan
B.P.lane
Bandra West
Bandra(east)
Bangur Nagar
Bazargate
Best Staff colony
Bharat Nagar
Bhawani Shankar
Bhawani Shankar rd
Borivali
Borivali East
Central Building
Century Mill
Chakala Midc
Chamarbaug
Charkop
Charni Road
Chinchbunder
Chinchpokli
Churchgate
Colaba
Cotton Exchange
Cumballa Hill
Dadar
Dadar Colony
Dahisar
Danda
Daulat Nagar
Delisle Road
Dharavi
Dharavi Road
Dockyard Road
Dr Deshmukh marg
Falkland Road
Girgaon
Gokhale Road
Goregaon
Goregaon East
Government Colony
Gowalia Tank
Grant Road
Haines Road
Hanuman Road
High Court bulding
Irla
Ins Hamla
International Airport
J.B. nagar
J.J.hospital
Jacob Circle
Jogeshwari East
Jogeshwari West
Juhu
Kalbadevi
Kamathipura
Kandivali East
Kandivali West
Ketkipada
Kharodi
Kherwadi
Kidwai Nagar
Lal Baug
Liberty Garden
M A marg
M.P.t.
Madh
Madhavbaug
Mahim
Mahim East
Malabar Hill
Malad
Malad East
Mandapeshwar
Mandvi
M

In [18]:
Mumbai_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agripada,18.975302,72.824898,Celejor,18.975844,72.823679,Bakery
1,Agripada,18.975302,72.824898,cafe coffee day,18.976988,72.824051,Coffee Shop
2,Agripada,18.975302,72.824898,HDFC Bank,18.973795,72.822895,Bank
3,Agripada,18.975302,72.824898,YMCA,18.972187,72.823491,Athletics & Sports
4,Agripada,18.975302,72.824898,YMCA Ground,18.972006,72.824011,Soccer Field


In [19]:
Mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agripada,6,6,6,6,6,6
Airport,5,5,5,5,5,5
Ambewadi,4,4,4,4,4,4
Andheri,8,8,8,8,8,8
Andheri East,13,13,13,13,13,13
...,...,...,...,...,...,...
Worli,10,10,10,10,10,10
Worli Colony,7,7,7,7,7,7
Worli Naka,8,8,8,8,8,8


Identifying the venue types in the neighbourhood! using One Hot method

In [20]:
Mumbai_onehot = pd.get_dummies(Mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mumbai_onehot['Neighborhood'] = Mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mumbai_onehot.columns[-1]] + list(Mumbai_onehot.columns[:-1])
Mumbai_onehot = Mumbai_onehot[fixed_columns]

Mumbai_onehot.head()

,Yoga Studio,ATM,Airport,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Camera Store,...,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Sushi Restaurant,Swiss Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Normalising the Data set

In [21]:
Mumbai_grouped = Mumbai_onehot.groupby('Neighborhood').mean().reset_index()
Mumbai_grouped

,Neighborhood,Yoga Studio,ATM,Airport,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,...,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Sushi Restaurant,Swiss Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Agripada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.166667,0.0,0.0,0.000000,0.0,0.166667,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166667,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.00
1,Airport,0.0,0.0,0.6,0.2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.00
2,Ambewadi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.25
3,Andheri,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.125000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.125000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.00
4,Andheri East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.076923,0.0,0.000000,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.076923,0.076923,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Worli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.300000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.00
130,Worli Colony,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.142857,0.0,0.0,0.000000,0.0,0.142857,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Identifying the top 5 venues in each neighbourhood!

In [22]:
num_top_venues = 5

for hood in Mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Mumbai_grouped[Mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agripada----
                venue  freq
0            Platform  0.17
1        Soccer Field  0.17
2         Coffee Shop  0.17
3  Athletics & Sports  0.17
4                Bank  0.17


----Airport----
                     venue  freq
0                  Airport   0.6
1                      Spa   0.2
2          Airport Service   0.2
3  North Indian Restaurant   0.0
4      Monument / Landmark   0.0


----Ambewadi----
                     venue  freq
0            Women's Store  0.25
1        Indian Restaurant  0.25
2         Department Store  0.25
3           Mattress Store  0.25
4  North Indian Restaurant  0.00


----Andheri----
                  venue  freq
0     Indian Restaurant  0.25
1  Fast Food Restaurant  0.25
2            Food Court  0.12
3        Sandwich Place  0.12
4            Restaurant  0.12


----Andheri East----
               venue  freq
0  Indian Restaurant  0.15
1           Gym Pool  0.08
2         Smoke Shop  0.08
3        Pizza Place  0.08
4      Shopping Mall  0.08

To arrange top venues in a neighbourhood 

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Mumbai_grouped['Neighborhood']

for ind in np.arange(Mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agripada,Soccer Field,Platform,Coffee Shop,Bank,Athletics & Sports,Bakery,Event Space,Food & Drink Shop,Food,Flower Shop
1,Airport,Airport,Spa,Airport Service,Event Space,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
2,Ambewadi,Women's Store,Indian Restaurant,Department Store,Mattress Store,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
3,Andheri,Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Bakery,Food Court,English Restaurant,Food,Flower Shop,Flea Market
4,Andheri East,Indian Restaurant,BBQ Joint,Chinese Restaurant,Smoke Shop,Food Truck,Bar,Camera Store,Electronics Store,Shopping Mall,Hotel
...,...,...,...,...,...,...,...,...,...,...,...
129,Worli,Scenic Lookout,Indian Restaurant,Garden,Paper / Office Supplies Store,Gun Range,Sports Club,Juice Bar,Women's Store,Flea Market,Fish & Chips Shop
130,Worli Colony,Soccer Field,Platform,Coffee Shop,Movie Theater,Bank,Athletics & Sports,Bakery,Falafel Restaurant,Food & Drink Shop,Food
131,Worli Naka,Coffee Shop,Bar,Modern European Restaurant,Spa,Vegetarian / Vegan Restaurant,Asian Restaurant,Train Station,Café,Falafel Restaurant,Farmers Market
132,Worli Police camp,Scenic Lookout,Sushi Restaurant,Arcade,Juice Bar,Garden,Café,Indian Restaurant,Art Gallery,Hotel,Paper / Office Supplies Store


Now using KMeans clustering algorithm to perform cluster analysis on the Data.

In [26]:
from sklearn.cluster import KMeans

kclusters = 4

Mumbai_grouped_clustering = Mumbai_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Mumbai_grouped_clustering)


For Clusers on map, we need to plot these neighbourhoods on folium and for that we need the Latitudes and Longitudes for the Neighbourhoods.

In [28]:
NegAdd = neighborhoods_venues_sorted['Neighborhood']

loc_latitude = []
loc_longitude = []

for myA in NegAdd:
  try:
        myA = myA + ", Mumbai"
        location = locator.geocode(myA)
        if location == None:
          Lat = 'NaN'
          Lng = 'NaN'
        else:
          Lat = location.latitude
          Lng = location.longitude
        loc_latitude.append(Lat)
        #print('loc_latitude',loc_latitude)
        loc_longitude.append(Lng)
  except Exception as e:
        print('error:::',e)

neighborhoods_venues_sorted['Latitude']= loc_latitude
neighborhoods_venues_sorted['Longitude']= loc_longitude    

Also Adding the Clustering Label in the Neighborhoods venue sorted Table

In [29]:
Mumbai_final = neighborhoods_venues_sorted

Mumbai_final['Cluster Labels'] = kmeans.labels_

Mumbai_final.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Cluster Labels
0,Agripada,Soccer Field,Platform,Coffee Shop,Bank,Athletics & Sports,Bakery,Event Space,Food & Drink Shop,Food,Flower Shop,18.975302,72.824898,0
1,Airport,Airport,Spa,Airport Service,Event Space,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.090131,72.863703,1
2,Ambewadi,Women's Store,Indian Restaurant,Department Store,Mattress Store,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.186776,72.859313,2
3,Andheri,Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Bakery,Food Court,English Restaurant,Food,Flower Shop,Flea Market,19.119698,72.846420,2
4,Andheri East,Indian Restaurant,BBQ Joint,Chinese Restaurant,Smoke Shop,Food Truck,Bar,Camera Store,Electronics Store,Shopping Mall,Hotel,19.115883,72.854202,0


Now plotting these clusters over Map to get insights!

In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mumbai_final['Latitude'], Mumbai_final['Longitude'], Mumbai_final['Neighborhood'], Mumbai_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

To Check the Cluster Information

In [31]:
Mumbai_final.loc[Mumbai_final['Cluster Labels'] == 0, Mumbai_final.columns[[1] + list(range(5, Mumbai_final.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Cluster Labels
0,Soccer Field,Athletics & Sports,Bakery,Event Space,Food & Drink Shop,Food,Flower Shop,18.975302,72.824898,0
4,Indian Restaurant,Food Truck,Bar,Camera Store,Electronics Store,Shopping Mall,Hotel,19.115883,72.854202,0
6,Bar,Women's Store,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,19.020761,72.865256,0
7,American Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field,18.900689,72.816134,0
8,Bar,Sandwich Place,Athletics & Sports,Asian Restaurant,Falafel Restaurant,Snack Place,Indian Restaurant,19.128315,72.840038,0
...,...,...,...,...,...,...,...,...,...,...
128,Pool,Women's Store,English Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.026919,72.875934,0
130,Soccer Field,Bank,Athletics & Sports,Bakery,Falafel Restaurant,Food & Drink Shop,Food,18.975226,72.823952,0
131,Coffee Shop,Vegetarian / Vegan Restaurant,Asian Restaurant,Train Station,Café,Falafel Restaurant,Farmers Market,18.986968,72.828547,0
132,Scenic Lookout,Garden,Café,Indian Restaurant,Art Gallery,Hotel,Paper / Office Supplies Store,19.009780,72.817244,0


In [32]:
Mumbai_final.loc[Mumbai_final['Cluster Labels'] == 1, Mumbai_final.columns[[1] + list(range(5, Mumbai_final.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Cluster Labels
1,Airport,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.090131,72.863703,1
55,Airport,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.090131,72.863703,1


In [33]:
Mumbai_final.loc[Mumbai_final['Cluster Labels'] == 2, Mumbai_final.columns[[1] + list(range(5, Mumbai_final.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Cluster Labels
2,Women's Store,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.186776,72.859313,2
3,Indian Restaurant,Bakery,Food Court,English Restaurant,Food,Flower Shop,Flea Market,19.119698,72.846420,2
5,Indian Restaurant,Bakery,Food Court,English Restaurant,Food,Flower Shop,Flea Market,19.119698,72.846420,2
12,Indian Restaurant,Café,Ice Cream Shop,Spa,Pizza Place,Restaurant,Fish & Chips Shop,19.061657,72.849811,2
14,Indian Restaurant,Coffee Shop,Restaurant,Irani Cafe,BBQ Joint,Parsi Restaurant,Hotel,18.933098,72.835860,2
17,Indian Restaurant,Women's Store,Chinese Restaurant,Breakfast Spot,Shopping Mall,Restaurant,Maharashtrian Restaurant,19.019009,72.840432,2
18,Indian Restaurant,Women's Store,Chinese Restaurant,Breakfast Spot,Shopping Mall,Restaurant,Maharashtrian Restaurant,19.019009,72.840432,2
20,Auto Garage,English Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,19.226723,72.861933,2
21,Indian Restaurant,Multiplex,Café,Athletics & Sports,Coffee Shop,Train Station,Hotel,18.938771,72.835335,2
24,Indian Restaurant,Coffee Shop,Clothing Store,Multicuisine Indian Restaurant,Restaurant,Roof Deck,Shoe Store,19.001759,72.840800,2


In [34]:
Mumbai_final.loc[Mumbai_final['Cluster Labels'] == 3, Mumbai_final.columns[[1] + list(range(5, Mumbai_final.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,Cluster Labels
9,Hotel,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,19.101937,72.861599,3


**Conclusion!**

Based on the above clusters, Cluster 3 seems to be a reasonable choice for a restaurant as there are less fast food restaurants in that area.